# Validate Uniform Density Field Grid Initialization

### Navigate to Root Directory

In [1]:
# Built-in modules
import os
import sys

# Insert package root to system path
cwd = os.getcwd()
parent_dir = cwd + "/../.."
sys.path.insert(1, parent_dir)

print("Directory containing the notebook:")
print(cwd)

Directory containing the notebook:
/home/users/jwakim/CodeDev/chromo_check_density_calculation/doc/tools


In [2]:
# Change working directory to package root
os.chdir(parent_dir)
print("Root Directory of Package: ")
print(os.getcwd())

Root Directory of Package: 
/home/users/jwakim/CodeDev/chromo_check_density_calculation


### Import Modules

In [3]:
# External modules
import numpy as np

# Package modules
from chromo.polymers import Chromatin
import chromo.binders
from chromo.fields import (
    UniformDensityField, get_blocks, get_neighboring_bins, assign_beads_to_bins
)

### Specify Reader Proteins

In [4]:
hp1 = chromo.binders.get_by_name("HP1")
binders = chromo.binders.make_binder_collection([hp1])

### Specify Confinement

In [5]:
confine_type = "Spherical"
confine_radius = 900

### Specify Polymer

In [6]:
num_beads = 100
bead_spacing = 16.5
chemical_mods = np.atleast_2d(np.ones(num_beads, dtype=int)).T
states = chemical_mods.copy()

polymer = Chromatin.confined_gaussian_walk(
    "Chr-1",
    num_beads,
    bead_spacing,
    states=states,
    confine_type=confine_type,
    confine_length=confine_radius,
    binder_names=np.array(['HP1']),
    chemical_mods=chemical_mods,
    chemical_mod_names=np.array(['H3K9me3'])
)

### Initialize Field

In [7]:
n_bins_x = 63
n_bins_y = 63
n_bins_z = 63

x_width = 2 * confine_radius
y_width = x_width
z_width = x_width

udf = UniformDensityField(
    polymers = [polymer],
    binders = binders,
    x_width = x_width,
    nx = n_bins_x,
    y_width = y_width,
    ny = n_bins_y,
    z_width = z_width,
    nz = n_bins_z,
    confine_type = confine_type,
    confine_length = confine_radius,
    chi = 0
)

### Check Bin Vertex Indices

In [8]:
for i in range(100):
    print(f"Bin Index: {i}")
    print(f"Corner Indices: {np.asarray(udf.bin_index[i])}")

Bin Index: 0
Corner Indices: [   0    1   63   64 3969 3970 4032 4033]
Bin Index: 1
Corner Indices: [   1    2   64   65 3970 3971 4033 4034]
Bin Index: 2
Corner Indices: [   2    3   65   66 3971 3972 4034 4035]
Bin Index: 3
Corner Indices: [   3    4   66   67 3972 3973 4035 4036]
Bin Index: 4
Corner Indices: [   4    5   67   68 3973 3974 4036 4037]
Bin Index: 5
Corner Indices: [   5    6   68   69 3974 3975 4037 4038]
Bin Index: 6
Corner Indices: [   6    7   69   70 3975 3976 4038 4039]
Bin Index: 7
Corner Indices: [   7    8   70   71 3976 3977 4039 4040]
Bin Index: 8
Corner Indices: [   8    9   71   72 3977 3978 4040 4041]
Bin Index: 9
Corner Indices: [   9   10   72   73 3978 3979 4041 4042]
Bin Index: 10
Corner Indices: [  10   11   73   74 3979 3980 4042 4043]
Bin Index: 11
Corner Indices: [  11   12   74   75 3980 3981 4043 4044]
Bin Index: 12
Corner Indices: [  12   13   75   76 3981 3982 4044 4045]
Bin Index: 13
Corner Indices: [  13   14   76   77 3982 3983 4045 4046]
Bi

### Check Super Indices

In [9]:
for i in range(10):
    x = i
    y = 0
    z = 0
    expected_super = i
    calculated_super = udf.inds_xyz_to_super[x, y, z]
    print(f"Expected Super Index: {expected_super}")
    print(f"Calculated Super Index: {calculated_super}\n")

for j in range(10):
    x = 0
    y = j
    z = 0
    expected_super = j * n_bins_x
    calculated_super = udf.inds_xyz_to_super[x, y, z]
    print(f"Expected Super Index: {expected_super}")
    print(f"Calculated Super Index: {calculated_super}\n")

for k in range(10):
    x = 0
    y = 0
    z = k
    expected_super = k * n_bins_x * n_bins_y
    calculated_super = udf.inds_xyz_to_super[x, y, z]
    print(f"Expected Super Index: {expected_super}")
    print(f"Calculated Super Index: {calculated_super}\n")

Expected Super Index: 0
Calculated Super Index: 0

Expected Super Index: 1
Calculated Super Index: 1

Expected Super Index: 2
Calculated Super Index: 2

Expected Super Index: 3
Calculated Super Index: 3

Expected Super Index: 4
Calculated Super Index: 4

Expected Super Index: 5
Calculated Super Index: 5

Expected Super Index: 6
Calculated Super Index: 6

Expected Super Index: 7
Calculated Super Index: 7

Expected Super Index: 8
Calculated Super Index: 8

Expected Super Index: 9
Calculated Super Index: 9

Expected Super Index: 0
Calculated Super Index: 0

Expected Super Index: 63
Calculated Super Index: 63

Expected Super Index: 126
Calculated Super Index: 126

Expected Super Index: 189
Calculated Super Index: 189

Expected Super Index: 252
Calculated Super Index: 252

Expected Super Index: 315
Calculated Super Index: 315

Expected Super Index: 378
Calculated Super Index: 378

Expected Super Index: 441
Calculated Super Index: 441

Expected Super Index: 504
Calculated Super Index: 504

E

### Check Confinement

In [10]:
print(f"Confinement Type: {udf.confine_type}")
print(f"Confinement Length: {udf.confine_length}")

Confinement Type: Spherical
Confinement Length: 900.0


### Check Accessible Volumes

In [11]:
access_vols = np.array(list(udf.access_vols.values()))
print(f"Unique Accessible Volumes: {np.unique(access_vols)}")

Unique Accessible Volumes: [ 2230.32069971  2241.98250729  2306.12244898 ... 20918.36734694
 20967.93002915 23323.61516035]


### Check Densities

In [12]:
densities = np.asarray(udf.density)
tot_densities = np.sum(densities, axis=0)
densities_trial = np.asarray(udf.density_trial)
tot_densities_trial = np.sum(densities_trial, axis=0)
bead_counts = np.multiply(densities[:, 0], access_vols)

print("Densities: ")
print(densities)

print("\nTotal Densities: ")
print(tot_densities)

print("\nDensities Trial: ")
print(densities_trial)

print("\nTotal Densities Trial (should be zero)")
print(tot_densities_trial)

print("\nBack Calculated Bead Count: ")
print(np.sum(bead_counts))

Densities: 
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]

Total Densities: 
[0.0042875 0.0042875]

Densities Trial: 
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]

Total Densities Trial (should be zero)
[0. 0.]

Back Calculated Bead Count: 
100.0
